Mainly serve as a template for future strategies. The template use simple moving average (SMA) of different timeframes to generate buy and sell signals. The higher timeframe SMA is used to generate the trend, and the lower timeframe SMA is used to generate the entry and exit points.

In [1]:
from typing_extensions import Dict
import optuna
import pandas as pd
import numpy as np
import talib as ta
# --- Our Imports ---
from AlgoTrade.Utils.DataManager import DataManager
from AlgoTrade.Config.BacktestConfig import BacktestConfig
from AlgoTrade.Backtester.BacktestRunner import BacktestRunner
from AlgoTrade.Factories.IndicatorFactory import IndicatorFactory
from AlgoTrade.Strat.Class.BaseStrategy import BaseStrategy
from AlgoTrade.Optimizer.StudyRunner import run_optimization
from AlgoTrade.Sizing.AtrBandsSizer import AtrBandsSizer
from AlgoTrade.Config.Enums import TradingMode

In [2]:
class MaDiffTimeframeStrategy(BaseStrategy):
    def __init__(
        self,
        config: BacktestConfig,
        symbol_ltf: str,
        tframe_ltf: str,
        symbol_htf: str,
        tframe_htf: str,
        start_date: str,
    ):
        super().__init__(config)
        self.dm = DataManager(name="bitget")
        self.symbol_ltf = symbol_ltf
        self.tframe_ltf = tframe_ltf
        self.symbol_htf = symbol_htf
        self.tframe_htf = tframe_htf
        self.start_date = start_date

    def generate_signals(self) -> pd.DataFrame:
        ltf_data = self.dm.from_local(self.symbol_ltf, self.tframe_ltf, self.start_date)
        htf_data = self.dm.from_local(self.symbol_htf, self.tframe_htf, self.start_date)

        ltf_factory = IndicatorFactory(ltf_data)
        df1 = ltf_factory.add_sma(90).add_stddev(14).add_atr(14).get_data()
        df1["SD_ATR_Spread"] = df1["STDDEV_14"] - df1["ATR_14"]

        htf_factory = IndicatorFactory(htf_data)
        df2 = htf_factory.add_sma(14).get_data()
        df2["trend"] = np.where(df2["close"] > df2["SMA_14"], 1, -1)
        df2["trend"] = pd.Series(df2["trend"]).shift(1).fillna(0)

        # Merge logic (simplified from original for clarity)
        df2_resampled = df2.reindex(df1.index, method="ffill")
        merged = df1.join(df2_resampled, rsuffix="_htf").dropna()

        signal = np.where(
            (merged["trend"] == 1) & (merged["close"] > merged["SMA_90"]),
            1,
            np.where(
                (merged["trend"] == -1) & (merged["close"] < merged["SMA_90"]), -1, 0
            ),
        )
        merged["signal"] = pd.Series(signal, index=merged.index).shift(1).fillna(0)
        return merged

In [3]:
leverage = 10
atr_sizer = AtrBandsSizer(
    risk_pct=0.01,  # Risk 1% of total equity per trade.
    atr_multiplier=1.0,  # Set Stop-Loss at 2x ATR away from the entry price.
    risk_reward_ratio=1.5,  # Set Take-Profit at 1.5x the Stop-Loss distance.
    leverage=leverage,  # Pass the leverage to the sizer.
)
config_atr_bands = BacktestConfig(
    initial_balance=3000.0,
    leverage=leverage,
    trading_mode=TradingMode.CROSS,
    sizing_strategy=atr_sizer,  # Pass the configured sizer object.
)

In [4]:
strategy = MaDiffTimeframeStrategy(
    config=config_atr_bands,
    symbol_ltf="ADA/USDT:USDT",
    tframe_ltf="15m",
    symbol_htf="ADA/USDT:USDT",
    tframe_htf="1h",
    start_date="2024-01-01",
)


In [5]:
analysis = strategy.run_single(generate_quantstats_report=True)

\n--- Running Single Backtest ---
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_15m.csv...
Load completed successfully. Data shape: (49120, 6)
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_1h.csv...


d:\ComputerScience\Trading\Quant2\AlgoTrade\Utils\DataManager.py:235: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data = pd.read_csv(file_path, index_col="date", parse_dates=True)


Load completed successfully. Data shape: (12756, 5)
Getting data...
Getting data...
Starting backtest...
Backtest finished.

--- Backtest Results ---
Period: [2024-01-01] -> [2025-05-26]
Initial Balance:        3,000.00
Final Balance:          0.00
ROI:                    -100.00%
Profit:                 11,159.13
Loss:                   14159.13
Sharpe Ratio:           -9.68
Sortino Ratio:          -14.19
Calmar Ratio:           -1.00
Max Drawdown:           100.00%
Total Trades:           10338
Total Good Trades:      4159
Total Bad Trades:       6179
Avg PnL Good Trades:    9.43%
Avg PnL Bad Trades:     -7.77%
Win Rate:               40.23%
Loss Rate:              59.77%
Profit Factor:          0.79
Generating QuantStats report to d:\ComputerScience\Trading\Quant2\quantstats\strategy_report.html...
HTML report saved to: d:\ComputerScience\Trading\Quant2\quantstats\strategy_report.html
Report saved to d:\ComputerScience\Trading\Quant2\quantstats\strategy_report.html


In [6]:
df = analysis.results_df
df.to_csv("results.csv")

In [5]:
df1, df2 = pre_process_data(ohlcv_data_s, ohlcv_data_l)
ltf_factory = IndicatorFactory(df1)
htf_factory = IndicatorFactory(df2)
df1 = ltf_factory.add_ema(20).add_ema(10).get_data()
df2 = htf_factory.add_ema(20).add_ema(10).get_data()

In [ ]:
merge_df = merge_htf_into_ltf(df1, df2, suffix="_4h")


In [ ]:
def compute_signals(merged_df: pd.DataFrame):
    signal = np.where(
        (merged_df["trend"] == 1)
        & (merged_df["close"] > merged_df["MA"])
        & (merged_df["SD_ATR_Spread"] > 0),
        1,
        np.where(
            (merged_df["trend"] == -1)
            & (merged_df["close"] < merged_df["MA"])
            & (merged_df["SD_ATR_Spread"] > 0),
            -1,
            0,
        ),
    )
    merged_df["signal"] = pd.Series(signal, index=merged_df.index).shift(1).fillna(0)
    return merged_df

In [ ]:
solution = compute_signals(merge_df)

# Set datetime to index to use the backtest

In [ ]:
solution.set_index('datetime', inplace=True)

# Select Position Sizing Method

In [ ]:
config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    position_sizing_method="PercentBalance",
    percent_balance_pct=0.1,  # <-- UPDATED
    exit_on_signal_0=True,
    # General SL/TP can be used as a fallback
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
)

In [ ]:
runner = BacktestRunner(config=config, data=solution)
analysis = runner.run()
analysis.print_metrics()

# Select All Position Sizing Method

In [ ]:
base_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    
    exit_on_signal_0=True,
    # --- Parameters for all sizing methods ---
    # For PercentBalance
    percent_balance_pct=0.1,
    # For FixedAmount
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
    # For AtrVolatility
    atr_volatility_risk_pct=0.02,
    atr_volatility_period=14,
    atr_volatility_multiplier=2.5,
    # For KellyCriterion
    kelly_criterion_lookback=50,
    kelly_criterion_fraction=0.5,
    # For AtrBands
    atr_bands_risk_pct=0.02,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
)

In [ ]:
from AlgoTrade.Backtester import ComparativeRunner as cr

In [ ]:
res = cr.run_comparative_analysis(base_config, solution)

In [ ]:
cr.print_comparison_report(res)

# Optimizer



In [ ]:
def generate_signals_for_trial(
    trial: optuna.trial.Trial, data_dict: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    """
    This function is passed to the optimizer. It defines the parameters to tune
    and returns a DataFrame with a 'signal' column.
    """
    # 1. Define the parameters you want to optimize for this strategy
    ltf_ma_period = trial.suggest_int("ltf_ma_period", 20, 100, step=5)
    htf_ma_period = trial.suggest_int("htf_ma_period", 5, 50, step=1)
    # You could also tune other things, e.g., a volatility filter threshold
    # min_spread = trial.suggest_float("min_spread", -0.0001, 0.0001)

    # 2. Get the data
    df1, df2 = pre_process_data(
        data_dict["ltf_data"].copy(), data_dict["htf_data"].copy()
    )

    # 3. Calculate indicators using the trial's parameters
    df1 = compute_indicators_df1(df1, period_df1=ltf_ma_period)
    df2 = compute_indicators_df2(df2, period_df2=htf_ma_period)

    # 4. Merge and compute final signal
    merged_df = merge_htf_into_ltf(df1, df2, suffix="_4h")
    solution_df = compute_signals(merged_df)

    # 5. Set index and return
    solution_df.set_index("datetime", inplace=True)
    return solution_df

In [ ]:
data_dictionary = {
    "ltf_data": ohlcv_data_s,
    "htf_data": ohlcv_data_l,
}

fixed_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    position_sizing_method="AtrBands",
    atr_bands_risk_pct=0.01,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
    exit_on_signal_0=False,
    allow_reverse_trade=False,
)

In [ ]:
print("\n--- Running Strategy Optimization ---")
run_optimization(
    data_dict=data_dictionary,
    config=fixed_config,
    strategy_function=generate_signals_for_trial,
    n_trials=10,
    metric_to_optimize="sharpe_ratio"
)
